In this notebook, we use the [method of manufactured solutions](http://prod.sandia.gov/techlib/access-control.cgi/2000/001444.pdf) to verify that [Phaseflow](https://github.com/geo-fluid-dynamics/phaseflow-fenics) correctly solves its governing equations.

# Set up this Jupyter notebook
Enable equation numbering:

In [1]:
%%javascript
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

Enable inline plotting with matplotlib:

In [2]:
%matplotlib inline

# Set up SymPy

In [17]:
import sympy
import sympy.vector
from sympy.vector import divergence as div, gradient as grad

t = sympy.symbols('t')

R = sympy.vector.CoordSys3D('R')

diff, exp, sin, tanh, transpose = sympy.diff, sympy.exp, sympy.sin, sympy.tanh, sympy.transpose

x, y, z = R.x, R.y, R.z

# Define the governing equations
The governing equations are

$$\begin{align}
    \nabla\cdot\mathbf{u} &= s_p, \\
    \frac{\partial}{\partial t}\mathbf{u} + \left(\mathbf{u}\cdot\nabla\right)\mathbf{u} + \nabla p - \nabla\cdot\left(2 \mu_{SL}\left((P(T)\right) \mathbf{D(u)}\right) 
    + \mathbf{f}_B(T) &= \mathbf{s_u}, \\
    \frac{\partial}{\partial t}(CT) + \nabla\cdot(CT\mathbf{u}) 
    - \mathscr{Pr}^{-1}\nabla\cdot(K\nabla T) + 
    \mathscr{Ste}^{-1} \frac{\partial}{\partial t} P(T) &= s_T
\end{align}$$

where

$$
\begin{align}
    \mathbf{D(u)} &= \frac{1}{2}\left(\nabla\mathbf{u} + \nabla\mathbf{u}^\intercal\right),\\
    P(T;T_f,r) &= \frac{1}{2}\left(1 - \tanh\frac{T_f - T}{r}\right),\\
    \mu_{SL}(P(T)) &= \mu_S + \left(\mu_L - \mu_S\right)P(T)
\end{align}
$$

In [30]:
dim = 3

def vector_gradient(u):
    """sympy.physics.vectors.gradient apparently does not handle vector fields, so we implement this."""
    gradu = sympy.Array(dim, 1)
    
    for i in range(dim):
    
        gradu[i] = grad(u[i])
    
    return gradu


def D(u):
    
    gradu = vector_gradient(u)
    
    graduT = transpose(gradu)
    
    return 0.5*(gradu + graduT)
    

def P_general(T, T_f):
    
    return 0.5*(1. - tanh((T_f - T)/r))


def mu_SL_general(PofT, mu_S, mu_L):
    
    return mu_S + (mu_L - mu_S)*PofT

though for this study we will only consider $T_f = 0.01, \mu_L = 1, \mu_S = 10^8$.

In [31]:
T_f = 0.01

def P(T):
    
    return P_general(T, T_f)



mu_S, mu_L = 1.e8, 1.

def mu_SL(PofT):
    
    return mu_SL_general(PofT, mu_S, mu_L)

Typically the right-hand sides are $\mathbf{s} = \left( s_p, \mathbf{s_u}, s_T \right)^\intercal = \mathbf{0}$, but we include $\mathbf{s}$ as a generic source term to facilitate code verification via MMS.

We denote the vector-valued solution of this system of PDE's as
$$\begin{align}
    \mathbf{w} = (p, \mathbf{u}, T)^\intercal
\end{align}$$

# Regularize
Note that we have already regularized the phase function with a hyperbolic tangent function. While testing Phaseflow so far, small smoothing parameters $r$ have destabilized the Newton solver. For the current MMS verification, we will use $r = 0.025$, the value from [the toy PCM melting regression test](https://github.com/geo-fluid-dynamics/phaseflow-fenics/blob/master/tests/test_melt_pcm.py).

In [32]:
r = 0.025

# Select a buoyancy model
For this study, we will only consider the linear buoyancy model from Danaila's simulation of the octadecane PCM melting benchmark, i.e.
$$ \begin{align}
    \mathbf{f}_B(T) = \left(0, \frac{-\mathscr{Ra}}{\mathscr{PrRe}^2}T\right)^\intercal
\end{align}$$

In [33]:
def f_B(T, Ra, Pr, Re):
    
    return -Ra/(Pr*Re*Re)*T*R.y

# Manufacture solution

To test the full capabilities of the implementation, we must select a solution where $\mathbf{u}$ and $T$ are twice differentiable in space and once differentiable in time, and where $p$ is once differentiable in space. For the present study, we will only consider a two-dimensional solution.

$$
\begin{align}
    p_\mathrm{MMS} &= e^{t} (\sin x + \sin{2y}), \\
    \mathbf{u}_\mathrm{MMS} &= e^{2t} (\sin{3x} + \sin{4y}, \sin{5x} + \sin{6y})^\intercal, \\
    T_\mathrm{MMS} &= e^{3t} (\sin{7x} + \sin{8y})
\end{align}
$$

In [34]:
p = exp(t)*(sin(x) + sin(2.*y))

u = exp(2.*t)*((sin(3.*x) + sin(4.*y))*R.i + (sin(5.*x) + sin(6.*y))*R.j)

T = exp(3.*t)*(sin(7.*x) + sin(8.*y))
    
# @todo Re-design the manufactured temperature solution such that it contains T_f
    
print("p = " + str(p))

print("u = " + str(u))

print("T = " + str(T))

p = (sin(R.x) + sin(2.0*R.y))*exp(t)
u = ((sin(3.0*R.x) + sin(4.0*R.y))*exp(2.0*t))*R.i + ((sin(5.0*R.x) + sin(6.0*R.y))*exp(2.0*t))*R.j
T = (sin(7.0*R.x) + sin(8.0*R.y))*exp(3.0*t)


# Derive the manufactured source term

Given the manufactured solution $\mathbf{w}_{\mathrm{MMS}}$, derive the manufactured source terms $\mathbf{s}_{\mathrm{MMS}}$ with SymPy.

In [35]:
s_p = div(u)

print("s_p = " + str(s_p))

s_p = 3.0*exp(2.0*t)*cos(3.0*R.x) + 6.0*exp(2.0*t)*cos(6.0*R.y)


The momentum equation is not as simple. SymPy does not view operators as tensors, and hence we cannot take the tensor product between the gradient operator and a tensor. Therefore, we must program this manually.

In [36]:
def nabla_dot_tensor(A):
    """ Compute $\nabla \dot \mathbf{A}$."""
    result = 0.
    
    for i in range(dim):
    
        for j, direction in enumerate([R.x, R.y, R.z]):
        
            result += A[i,j].diff(R[i], R)*direction
    
    return result

In [37]:
s_u = u.diff(t)

s_u += sympy.tensorproduct(vector_gradient(u), u) 

s_u += grad(p) 

s_u -= 2.*nabla_dot_tensor(mu_SL(P(T))*D(u)) 

s_u += f_B(T)

print("s_u = " + str(s_u))

TypeError: 'int' object is not iterable

In [ ]:
import fenics

In [ ]:
import phaseflow